In [1]:
from gplearn.genetic import SymbolicTransformer
import pandas as pd
import numpy as np

In [2]:
train_X = pd.read_parquet('/home/soheil/Soheil/lin-python/S/dsa2/data/output/airbnb/airbnb_lightgbm/features_store/colnum/features.parquet')

In [3]:
train_y = pd.read_parquet('/home/soheil/Soheil/lin-python/S/dsa2/data/output/airbnb/airbnb_lightgbm/features_store/dfy/features.parquet')

,price
id,
21869,70.0
151249,81.0
29279,130.0
38296,80.0
342637,65.0
...,...
242927,95.0
229075,75.0
189288,100.0


In [23]:
train_X = train_X.fillna(method='ffill')

In [13]:
train_X.columns.to_list()

['review_scores_communication',
 'review_scores_location',
 'review_scores_rating',
 'review_scores_communication_bin',
 'review_scores_location_bin',
 'review_scores_rating_bin']

In [24]:
train_y

,price
id,
21869,70.0
151249,81.0
29279,130.0
38296,80.0
342637,65.0
...,...
242927,95.0
229075,75.0
189288,100.0


In [58]:
from gplearn.functions import make_function

def squaree(x) :  return x*x
    
square_ = make_function(function=squaree,
                        name='square_',
                        arity=1)


function_set = ['add', 'sub', 'mul', 'div',  'sqrt', 'log', 'abs', 'neg', 'inv','tan', square_]
pars_genetic = { 'generations' : 2, 'population_size' : 10,  
                 'tournament_size' : 20, 'stopping_criteria' : 1.0, 'const_range' : (-1.,1.),
                 'p_crossover' : 0.9, 'p_subtree_mutation' : 0.01, 'p_hoist_mutation' : 0.01,
                 'p_point_mutation' : 0.01, 'p_point_replace' : 0.05} 

feature_name_ = [f'X{ii_}' for ii_ in range(train_X.shape[1])]

# map_dict_X = dict(zip(train_X.columns.to_list(), feature_name_))
# map_dict_y = 

gp = SymbolicTransformer(hall_of_fame=train_X.shape[1]+1,
                        n_components=train_X.shape[1],
                        feature_names=feature_name_,
                        function_set=function_set,
                        parsimony_coefficient=0.0005,
                        max_samples=0.9, verbose=1,
                        random_state=0, n_jobs=6, **pars_genetic)

In [59]:
gp.fit(train_X, train_y)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    11.80         0.055381        7         0.108044        0.0481559      0.10s
   1     7.10        0.0792887        8         0.123538         0.141358      0.00s


/home/soheil/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SymbolicTransformer(feature_names=['X0', 'X1', 'X2', 'X3', 'X4', 'X5'],
                    function_set=['add', 'sub', 'mul', 'div', 'sqrt', 'log',
                                  'abs', 'neg', 'inv', 'tan',
                    generations=2, hall_of_fame=7, max_samples=0.9,
                    n_components=6, n_jobs=6, parsimony_coefficient=0.0005,
                    population_size=10, random_state=0, verbose=1)

In [44]:
aaa = gp[0]

In [60]:
print(gp)

[log(square_(square_(neg(neg(sub(X5, X0)))))),
 log(log(square_(neg(neg(sub(X5, X0)))))),
 log(square_(sub(X5, X0))),
 log(square_(neg(square_(neg(neg(sub(X5, X0))))))),
 log(square_(neg(neg(sub(X5, X0))))),
 log(square_(add(X4, tan(sqrt(log(X4))))))]


In [7]:
from sympy import sympify

In [62]:
gp.transform(train_X)

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        5.07188629],
       [6.43775165, 1.16903218, 3.21887582, 6.43775165, 3.21887582,
        6.01837048],
       [2.77258872, 0.32663426, 1.38629436, 2.77258872, 1.38629436,
        6.01837048],
       ...,
       [4.39444915, 0.78719501, 2.19722458, 4.39444915, 2.19722458,
        5.07188629],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        5.07188629],
       [4.39444915, 0.78719501, 2.19722458, 4.39444915, 2.19722458,
        3.71557148]])

In [32]:
a

[X1*X2]

In [29]:
converter = {
    'sub': lambda x, y : x - y,
    'div': lambda x, y : x/y,
    'mul': lambda x, y : x*y,
    'add': lambda x, y : x + y,
    'neg': lambda x    : -x,
    'pow': lambda x, y : x**y
}
a = sympify(str(gp), locals=converter)

In [30]:
str(a)

'[X1*X2]'

In [36]:
import pickle
with open('formula.pkl','wb') as f_:
    pickle.dump(str(a),f_)

In [6]:
pd.read_parquet('/home/soheil/Soheil/lin-python/S/dsa2/data/output/airbnb/airbnb_lightgbm/check/dfX.parquet')

,id,price,review_scores_communication,review_scores_location,review_scores_rating,cancellation_policy,host_response_rate,host_response_time,review_scores_communication_bin,review_scores_location_bin,...,host_response_rate_int_0-review_scores_communication_bin_8,host_response_rate_int_0-review_scores_communication_bin_9,host_response_rate_int_0-review_scores_location_bin_7,host_response_rate_int_0-review_scores_location_bin_9,review_scores_communication_bin_-1-review_scores_location_bin_-1,review_scores_communication_bin_8-review_scores_location_bin_7,review_scores_communication_bin_8-review_scores_location_bin_9,review_scores_communication_bin_9-review_scores_location_bin_7,review_scores_communication_bin_9-review_scores_location_bin_9,price_pred
0,248192,0.49,9.0,9.0,87.0,strict_14_with_grace_period,100%,within an hour,8,7,...,1,0,1,0,0,1,0,0,0,0.676620
1,212348,0.55,NaN,NaN,NaN,moderate,100%,within an hour,-1,-1,...,0,0,0,0,1,0,0,0,0,0.673326
2,87549,0.48,10.0,10.0,94.0,strict_14_with_grace_period,None,None,9,9,...,0,0,0,0,0,0,0,0,1,0.673090
3,192052,0.79,10.0,10.0,96.0,strict_14_with_grace_period,100%,within an hour,9,9,...,0,1,0,1,0,0,0,0,1,0.733971
4,251398,0.40,10.0,9.0,97.0,moderate,100%,within a few hours,9,7,...,0,1,1,0,0,0,0,1,0,0.689654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,250023,0.70,9.0,9.0,89.0,moderate,None,None,8,7,...,0,0,0,0,0,1,0,0,0,0.675373
285,251548,0.55,10.0,10.0,93.0,strict_14_with_grace_period,100%,within an hour,9,9,...,0,1,0,1,0,0,0,0,1,0.733971
286,72079,0.80,10.0,10.0,97.0,moderate,None,None,9,9,...,0,0,0,0,0,0,0,0,1,0.665591
287,83568,0.70,10.0,10.0,100.0,strict_14_with_grace_period,100%,within a few hours,9,9,...,0,1,0,1,0,0,0,0,1,0.737916
